## Importing

In [ ]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 27.2 MB/s eta 0:00:00
  Using cache

In [ ]:
# @title
import textattack
import transformers
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## BAE: BERT-based Adversarial Examples for Text Classification

In [ ]:
# Load model, tokenizer, and model_wrapper
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("imdb", None, "test", shuffle=True)

textattack: Loading datasets dataset imdb, split test.


In [ ]:
attack = BAEGarg2019.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=20, parallel = True)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 




  5%|▌         | 1/20 [01:06<21:12, 66.95s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   5%|▌         | 1/20 [01:06<21:12, 66.96s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. 


[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 2/20 [01:12<10:56, 36.48s/it]

...

--------------------------------------------- Result 17 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (60%)]]

[[This]] cheap, [[grainy-filmed]] Italian flick is about a couple of [[inheritors]] of a manor in the Italian countryside who head up to the house to [[stay]], and then find themselves getting killed off by ghosts of people killed in that [[house]].<[[br]] /><br />I wasn't impressed by this. It wasn't really that scary, mostly just the [[way]] a [[cheap]] Italian film should be. A girl, her two [[cousins]], and one cousin's girlfriend, head to this huge house for some reason (I couldn't figure out why) and are [[staying]] there, cleaning up and checking out the [[place]]. Characters come in and out of the film, and it's quite boring at points, and the [[majority]] of deaths are quite rushed. The girlfriend is [[hit]] by a car when fleeing the house after having a dream of her death, and the scene is quite good, but then [[things]] get slow ag


[Succeeded / Failed / Skipped / Total] 11 / 5 / 1 / 17:  90%|█████████ | 18/20 [05:38<00:37, 18.80s/it]
[Succeeded / Failed / Skipped / Total] 12 / 5 / 1 / 18:  90%|█████████ | 18/20 [05:38<00:37, 18.80s/it]
[Succeeded / Failed / Skipped / Total] 12 / 5 / 2 / 19:  95%|█████████▌| 19/20 [05:38<00:17, 17.82s/it]

--------------------------------------------- Result 18 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (84%)]]

I just finished watching this movie and am disappointed to say that I didn't enjoy it a bit. It is so slow Slow and [[uninteresting]]. This kid from Harry Potter plays a shy teenager with an rude mother, and then one day the rude mother tells the kid to find a job so that they could accommodate an old guy apparently having no place to live has started to live with his family and therefore the kid goes to work for a old lady. And this old lady who is living all alone teaches him about girls, driving car and life! I couldn't get how an 18 year old guy enjoy spending time with an awful lady in her 80s. Sorry if my comments on this movie has bothered people who might have enjoyed it, I could be wrong as I am not British and may not understand the social and their family structure and way of life. Mostly the movie is made for the British [[audienc


[Succeeded / Failed / Skipped / Total] 12 / 5 / 2 / 19: 100%|██████████| 20/20 [06:04<00:00, 18.24s/it]
[Succeeded / Failed / Skipped / Total] 13 / 5 / 2 / 20: 100%|██████████| 20/20 [06:04<00:00, 18.24s/it]

--------------------------------------------- Result 20 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (53%)]]

Every movie I have PPV'd because Leonard Maltin praised it to the skies has blown chunks! Every single one! When will I ever [[learn]]?<br /><br />Evie is a raving Old Bag who thinks nothing of saying she's dying of breast cancer to get her way! Laura is an insufferable Medusa filled with The Holy Spirit (and her hubby's protégé)! [[Caught]] between these harpies is Medusa's dumb-as-a-rock boy who has been pressed into weed-pulling servitude by The Old Bag!<br /><br />As I [[said]], when will I ever learn?<br /><br />I was temporarily lifted out of my malaise when The Old Bag stuck her head in a sink, but, unfortunately, she did not die. I was temporarily lifted out of my malaise again when Medusa got mowed down, but, unfortunately, she did not die. It should be a capital offense to torture audiences like this!<br /><br />Without Harry Potter

### Try with yelp dataset

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("yelp_polarity", None, "test", shuffle=True)

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
attack = BAEGarg2019.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10, parallel = True)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [00:25<03:51, 25.75s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [00:25<03:51, 25.75s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (100%)]] --> [[[FAILED]]]

Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.





[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  20%|██        | 2/10 [00:31<02:07, 15.93s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:31<02:07, 15.94s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (100%)]] --> [[2 (56%)]]

Ok! [[Let]] me [[tell]] you about my bad experience first. I went to [[D]]&[[B]] last [[night]] for a post wedding party - which, side note, is a great [[idea]]!\n\nIt was around midnight and the [[bar]] wasn't [[really]] populated. There were three bartenders and only one was actually [[making]] rounds to [[see]] if anyone needed anything. The two other bartenders were [[chatting]] on the far side of the bar that no [[one]] was sitting at. Kind of counter productive if you [[ask]] me. \n\nI stood there for about [[5]] minutes, which for a busy bar is fine but when [[I]] am the only [[one]] with my card out then, it just seems a little ridiculous. I made eye contact with the one girl [[twice]] and gave her a smile and she literally turned away. I finally had to walk to them to get their attention.  I was standing right in front of them smiling and they didn


[Succeeded / Failed / Skipped / Total] 4 / 4 / 0 / 8:  90%|█████████ | 9/10 [02:33<00:17, 17.09s/it]
[Succeeded / Failed / Skipped / Total] 4 / 5 / 0 / 9:  90%|█████████ | 9/10 [02:33<00:17, 17.10s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (100%)]] --> [[[FAILED]]]

I used to love D&B when it first opened in the Waterfront, but it has gone down hill over the years. The games are not as fun and do not give you as many tickets and the prizes have gotten cheaper in quality. It takes a whole heck of a lot of tickets for you to even get a pencil! The atmosphere is okay but it used to be so much better with the funnest games and diverse groups of people! Now, it is run down and many of the games are app related games (Fruit Ninja) and 3D Experience rides. With such \""games\"", you can't even earn tickets and they take a lot of tokens! Last time I went, back in the winter, many of the games were broken, which made for a negative player experience. I would go to D&B to play some games again in the future, but it is no longer one of my favorite places to go due to the decline of fun games where you can earn tickets.





[Succeeded / Failed / Skipped / Total] 4 / 5 / 0 / 9: 100%|██████████| 10/10 [02:44<00:00, 16.42s/it]
[Succeeded / Failed / Skipped / Total] 4 / 6 / 0 / 10: 100%|██████████| 10/10 [02:44<00:00, 16.42s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[2 (100%)]] --> [[[FAILED]]]

Like any Barnes & Noble, it has a nice comfy cafe, and a large selection of books.  The staff is very friendly and helpful.  They stock a decent selection, and the prices are pretty reasonable.  Obviously it's hard for them to compete with Amazon.  However since all the small shop bookstores are gone, it's nice to walk into one every once in a while.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 60.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 5.79%  |
| Average num. words per input: | 121.2  |
| Avg num queries:              | 201.

### Try AD news dataset

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
ag_news = HuggingFaceDataset("ag_news", None, "test", shuffle=True)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
attack = BAEGarg2019.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10, parallel=True)
attacker = Attacker(attack, ag_news, attack_args)
attacker.attack_dataset()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

--------------------------------------------- Result 9 ---------------------------------------------
[[Sci/tech (98%)]] --> [[[FAILED]]]

E-mail scam targets police chief Wiltshire Police warns about "phishing" after its fraud squad chief was targeted.


[Succeeded / Failed / Skipped / Total] 2 / 8 / 0 / 10: 100%|██████████| 10/10 [03:06<00:00, 18.63s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Sci/tech (98%)]] --> [[Business (93%)]]

Card fraud unit nets 36,000 cards In its first two years, the [[UK's]] dedicated card fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved [[65m]].

Card fraud unit nets 36,000 cards In its first two years, the [[london]] dedicated card fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved [[billion]].


[Succeeded / Failed / Skipped / Total] 2 / 8 / 0 / 10: 100%|██████████| 10/10 [03:09<00:00, 18.95s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 20.0%  |
| Average perturbed word %:     | 7.12%  |
| Average num. words per input: | 63.0   |
| Avg num queries:              | 185.4  |
+-------------------------------+--------+


## DeepWordBug

In [ ]:
from textattack.attack_recipes import DeepWordBugGao2018

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Load dataset
dataset = HuggingFaceDataset("yelp_polarity", None, "test", shuffle=True)

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
attack = DeepWordBugGao2018.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [00:02<00:25,  2.82s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:02<00:25,  2.84s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (100%)]] --> [[1 (52%)]]

Contrary to other reviews, I have zero [[complaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and [[know]] what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. [[But]] here, my service and road coverage has all been [[well]] explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary to other reviews, I have zero [[complants]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my e


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 2/10 [00:06<00:26,  3.30s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:06<00:26,  3.31s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (100%)]] --> [[2 (79%)]]

Ok! Let me tell you about my [[bad]] experience first. I went to D&B last night for a post wedding party - which, side note, is a great [[idea]]!\n\nIt was around midnight and the bar wasn't really populated. There were three bartenders and only one was actually making rounds to see if anyone needed anything. The two other bartenders were chatting on the far side of the bar that no one was sitting at. Kind of counter productive if you ask me. \n\nI stood there for about 5 minutes, which for a busy bar is fine but when I am the only one with my card out then, it just seems a little ridiculous. I made eye contact with the one girl twice and gave her a smile and she literally turned away. I finally had to walk to them to get their attention.  I was standing right in front of them smiling and they didn't ask if i need anything. I finally said, \""Are you workin


[Succeeded / Failed / Skipped / Total] 7 / 1 / 0 / 8:  90%|█████████ | 9/10 [00:52<00:05,  5.78s/it]
[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [00:52<00:05,  5.78s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (100%)]] --> [[[FAILED]]]

I used to love D&B when it first opened in the Waterfront, but it has gone down hill over the years. The games are not as fun and do not give you as many tickets and the prizes have gotten cheaper in quality. It takes a whole heck of a lot of tickets for you to even get a pencil! The atmosphere is okay but it used to be so much better with the funnest games and diverse groups of people! Now, it is run down and many of the games are app related games (Fruit Ninja) and 3D Experience rides. With such \""games\"", you can't even earn tickets and they take a lot of tokens! Last time I went, back in the winter, many of the games were broken, which made for a negative player experience. I would go to D&B to play some games again in the future, but it is no longer one of my favorite places to go due to the decline of fun games where you can earn tickets.





[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9: 100%|██████████| 10/10 [00:54<00:00,  5.49s/it]
[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [00:54<00:00,  5.49s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[2 (100%)]] --> [[1 (86%)]]

Like any Barnes & Noble, it has a nice [[comfy]] cafe, and a large selection of books.  The [[staff]] is very [[friendly]] and helpful.  [[They]] stock a decent selection, and the prices are pretty reasonable.  Obviously it's hard for them to compete with Amazon.  [[However]] [[since]] all the small shop [[bookstores]] are gone, it's [[nice]] to [[walk]] into one [[every]] once in a while.

Like any Barnes & Noble, it has a nice [[coxfy]] cafe, and a large selection of books.  The [[stanf]] is very [[frieddly]] and helpful.  [[The]] stock a decent selection, and the prices are pretty reasonable.  Obviously it's hard for them to compete with Amazon.  [[Howeevr]] [[isnce]] all the small shop [[bokostores]] are gone, it's [[nLce]] to [[wlk]] into one [[eveyr]] once in a while.



+-------------------------------+--------+
| Attack Results                |    

## Simulated Annealing

In [ ]:
from textattack.search_methods import SearchMethod
from textattack.goal_function_results import GoalFunctionResultStatus
from textattack.constraints.pre_transformation import RepeatModification
import random
import numpy as np

In [ ]:
class SimulatedAnnealing(SearchMethod):

    def __init__(self, max_temp=100.0, max_iter=10, K=10, radius=3, delta=0.01):
        self.max_temp = max_temp
        self.max_iter = max_iter
        self.K = K
        self.radius = radius
        self.delta = delta

    def _get_score(self, original_text, adversarial_text):
        result, search_over = self.get_goal_results([adversarial_text])
        score = result[0].score
        cost = len(original_text.all_words_diff(adversarial_text))
        y = score - (self.delta * cost)
        return y, search_over, result[0]


    def _get_transformations(self, indices, current_text, initial_result):
        transformations = []
        scores = []
        curr_result = initial_result
        for i in indices:
            t_i = self.get_transformations(
                    current_text,
                    original_text=initial_result.attacked_text,
                    indices_to_modify=[i])
            if len(t_i) == 0:
                transformations.append(current_text.words[i])
                scores.append(curr_result.score)
                continue

            res_i, _ = self.get_goal_results(t_i)
            scores_i = np.array([r.score for r in res_i])
            max_score_index = np.argmax(scores_i)
            curr_result = res_i[max_score_index]
            scores.append(scores_i[max_score_index])
            transformations.append(t_i[max_score_index].words[i])
        return np.array(transformations), np.array(scores)

    def perform_search(self, initial_result):
        best_result = initial_result
        current_text = initial_result.attacked_text
        max_temp = self.max_temp
        t = 0

        _, indices = self.get_indices_to_order(current_text)
        transformations, scores = self._get_transformations(indices, current_text, initial_result)
        transformations = list(transformations[(-scores).argsort()])
        indices = np.array(indices)[(-scores).argsort()]
        n = len(transformations)
        if n == 0:
            return best_result

        while not best_result.goal_status == GoalFunctionResultStatus.SUCCEEDED:
            if max_temp == 0.0 or t == self.max_iter:
                return best_result

            #Internal simulations
            for k in range(self.K):
                index = t + random.randint(0, t*self.radius)
                if index >= n:
                    continue
                original_index = indices[index]
                word_adv = transformations[index]
                next_text = current_text.replace_word_at_index(original_index, word_adv)

                curr_score = best_result.score
                next_score, search_over, result = self._get_score(initial_result.attacked_text, next_text)
                if search_over:
                    return result
                delta = next_score - curr_score
                if (delta > 0) or (random.random() < np.exp(delta / max_temp)):
                    current_text = next_text
                    best_result = result
            t += 1
            #Annealing
            max_temp = max_temp / (t + 1)

        return best_result


    @property
    def is_black_box(self):
        return True

    def extra_repr_keys(self):
        return ["max_temp", "max_iter", "K", "radius", "delta"]

In [ ]:
from textattack.attack_recipes.attack_recipe import AttackRecipe
from textattack.transformations import WordSwapMaskedLM
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.grammaticality import PartOfSpeech
from textattack import Attack

In [ ]:
class BESAYang2020(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        transformation = WordSwapMaskedLM(
            method="bae", max_candidates=50
        )
        constraints = []
        constraints.append(PartOfSpeech())
        use_constraint = UniversalSentenceEncoder(
            threshold=0.5,
            metric="cosine",
            compare_against_original=True,
            window_size=15
        )
        constraints.append(use_constraint)

        goal_function = UntargetedClassification(model_wrapper)
        search_method = SimulatedAnnealing()

        return Attack(goal_function, constraints, transformation, search_method)




In [ ]:
attack = BESAYang2020.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10)
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): SimulatedAnnealing(
    (max_temp):  100.0
    (max_iter):  10
    (K):  10
    (radius):  3
    (delta):  0.01
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.5
        (window_size):  15
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
  (is_black_box):  True
) 




  0%|          | 0/10 [05:27<?, ?it/s]

 10%|█         | 1/10 [01:50<16:35, 110.57s/it]

--------------------------------------------- Result 1 ---------------------------------------------



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [01:50<16:38, 110.93s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (100%)]] --> [[2 (85%)]]

Ok! Let me tell you about my [[bad]] experience first. I went to D&B last night for a post wedding party - which, side note, is a great [[idea]]!\n\nIt was around midnight and the bar wasn't really populated. There were three bartenders and only one was actually making rounds to see if anyone needed anything. The two other bartenders were chatting on the far side of the bar that no one was sitting at. Kind of counter productive if you ask me. \n\nI stood there for about 5 minutes, which for a busy bar is fine but when I am the only one with my card out then, it just seems a little ridiculous. I made eye contact with the one girl twice and gave her a smile and she literally turned away. I finally had to walk to them to get their attention.  I was standing right in front of them smiling and they didn't ask if i need anything. I finally said, \""Are you workin


[Succeeded / Failed / Skipped / Total] 6 / 2 / 0 / 8:  90%|█████████ | 9/10 [14:16<01:35, 95.22s/it]
[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [14:16<01:35, 95.22s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (100%)]] --> [[2 (96%)]]

I used to love D&B when it first opened in the Waterfront, but it has [[gone]] down [[hill]] over the years. The games are not as fun and do not give you as many tickets and the prizes have gotten cheaper in quality. It takes a whole heck of a lot of tickets for you to even get a pencil! The atmosphere is okay but it used to be so much better with the funnest games and diverse groups of people! Now, it is [[run]] [[down]] and many of the games are app related games (Fruit Ninja) and 3D Experience rides. With such \""games\"", you can't even earn tickets and they take a lot of tokens! Last time I went, back in the winter, many of the games were broken, which made for a [[negative]] player experience. I would [[go]] to D&B to play some games again in the future, but it is no longer one of my favorite places to go due to the decline of fun games where you can 


[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9: 100%|██████████| 10/10 [15:01<00:00, 90.16s/it]
[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [15:01<00:00, 90.16s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[2 (100%)]] --> [[1 (56%)]]

Like any Barnes & Noble, it has a nice comfy cafe, and a [[large]] selection of books.  The staff is very friendly and helpful.  They stock a decent selection, and the prices are pretty [[reasonable]].  Obviously it's hard for them to compete with Amazon.  However since all the small shop bookstores are gone, it's nice to walk into one [[every]] once in a while.

Like any Barnes & Noble, it has a nice comfy cafe, and a [[limited]] selection of books.  The staff is very friendly and helpful.  They stock a decent selection, and the prices are pretty [[ridiculous]].  Obviously it's hard for them to compete with Amazon.  However since all the small shop bookstores are gone, it's nice to walk into one [[another]] once in a while.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
|

### Try ag news

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
ag_news = HuggingFaceDataset("ag_news", None, "test", shuffle=True)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


In [ ]:
attack = BESAYang2020.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=10)
attacker = Attacker(attack, ag_news, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): SimulatedAnnealing(
    (max_temp):  100.0
    (max_iter):  10
    (K):  10
    (radius):  3
    (delta):  0.01
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.5
        (window_size):  15
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:33<05:03, 33.69s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (68%)]]

Fears for T [[N]] pension after talks Unions representing [[workers]] at Turner   [[Newall]] say they are 'disappointed' after talks with stricken [[parent]] firm [[Federal]] Mogul.

Fears for T [[pl]] pension after talks Unions representing [[australia]] at Turner   [[sports]] say they are 'disappointed' after talks with stricken [[software]] firm [[software]] Mogul.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [01:14<04:59, 37.46s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

FOAFKey: FOAF, PGP, Key Distribution, and Bloom Filters \\FOAF/LOAF  and bloom filters have a lot of interesting properties for social\network and whitelist distribution.\\I think we can go one level higher though and include GPG/OpenPGP key\fingerpring distribution in the FOAF file for simple web-of-trust based key\distribution.\\What if we used FOAF and included the PGP key fingerprint(s) for identities?\This could mean a lot.  You include the PGP key fingerprints within the FOAF\file of your direct friends and then include a bloom filter of the PGP key\fingerprints of your entire whitelist (the source FOAF file would of course need\to be encrypted ).\\Your whitelist would be populated from the social network as your client\discovered new identit ...\\




[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [08:36<00:57, 57.40s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (98%)]]

E-mail scam [[targets]] police chief Wiltshire Police warns about "[[phishing]]" after its [[fraud]] [[squad]] chief was [[targeted]].

E-mail scam [[bbc]] police chief Wiltshire Police warns about "[[corruption]]" after its [[bomb]] [[campaign]] chief was [[hanged]].




[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [08:58<00:00, 53.85s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Sci/tech (98%)]] --> [[Business (89%)]]

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[card]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[cash]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 15.63% |
| Average num. words per input: | 63.0   |
| Avg num queries:    